# This notebook has definitions and functions used in common by notebooks using the wikibase API

The API documentation is [here](https://www.wikidata.org/w/api.php?action=help&modules=wbeditentity)

To load these functions into another notebook, do `%run WB_common.ipynb` inside a code cell, e.g. once at the top of the page.

In [ ]:
import pandas as pd

## Subclass Exception to define custom exception for wikibase API errors

In [1]:
class WBAPIException(BaseException):
    """Raised when the wikibase Open API throws an error"""
    pass

## Login to the Wikibase

In [1]:
import requests
import json 
import configparser

def login(username, botpwd):
    """
    Starts a new session and logins using a bot account.
    @username, @botpwd string: credentials of an existing bot user
    @returns requests.sessions.Session object
    """
    # create a new session
    session = requests.Session()

    # get login token
    r1 = session.get(WIKIBASE_API, params={
        'format': 'json',
        'action': 'query',
        'meta': 'tokens',
        'type': 'login'
    })
    # login with bot account
    r2 = session.post(WIKIBASE_API, data={
        'format': 'json',
        'action': 'login',
        'lgname': username,
        'lgpassword': botpwd,
        'lgtoken': r1.json()['query']['tokens']['logintoken'],
    })
    # raise when login failed
    if r2.json()['login']['result'] != 'Success':
        raise WBAPIException(r2.json()['login'])
        
    return session

## Create a wikibase property
A function that creates a new wikidatabase property and returns the new id.

Properties can be seen in the wiki under `http:localhost:8080/wiki/Property:Px`

In [4]:
def get_csrf_token(session):
    """Gets a security (CSRF) token."""
    params1 = {
        "action": "query",
        "meta": "tokens",
        "type": "csrf"
    }
    r1 = session.get(WIKIBASE_API, params=params1)
    token = r1.json()['query']['tokens']['csrftoken']

    return token
    

def create_property(session, data):
    """
    Creates a wikibase property.
    @session requests.sessions.Session: session obtained from login 
    @data python dict: creation parameters of the property
    @returns string: id of the new property
    """
    token = get_csrf_token(session)
    
    params = {
        "action": "wbeditentity",
        "format": "json",
        'new': 'property',
        'data': json.dumps(data),
        'token': token
    }
    r1 = session.post(WIKIBASE_API, data=params)
    r1.json = r1.json()
    
    # raise when edit failed
    if 'error' in r1.json.keys():
        raise WBAPIException(r1.json['error'])

    return r1.json['entity']['id']

## Create a wikibase entity
A function that creates a new wikidatabase entity (item) and returns the new id.

If the property label already exists in the wiki, will not overwrite it, but return the existing QID ([example](https://query.portal.mardi4nfdi.de/embed.html#PREFIX%20wdt%3A%20%3Chttps%3A%2F%2Fportal.mardi4nfdi.de%2Fprop%2Fdirect%2F%3E%0APREFIX%20wd%3A%20%3Chttps%3A%2F%2Fportal.mardi4nfdi.de%2Fentity%2F%3E%0A%0ASELECT%20%20%3FqId%20%3FqIdLabel%20%3FswmathID%20%0AWHERE%20%7B%0A%20%20%3FqId%20wdt%3AP13%20%3FswmathID%20.%20%20%20%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22en%22.%20%7D%0A%20%20%20%20%20%20%7D%0ALimit%2010)).

Items can be seen in the wiki under `http:localhost:8080/wiki/Item:Qx`

In [5]:
def create_entity(session, data):
    """
    Creates a wikibase entity.
    @session requests.sessions.Session: session obtained from login 
    @data python dict: creation parameters of the entity
    @returns string: id of the new entity
    """
    token = get_csrf_token(session)
    
    params = {
        'action': 'wbeditentity',
        'format': 'json',
        'new': 'item',
        'data': json.dumps(data),
        'token': token
    }
    r1 = session.post(WIKIBASE_API, data=params)
    r1.json = r1.json()
    
    # raise when edit failed
    if 'error' in r1.json.keys():
        raise WBAPIException(r1.json['error'])

    return r1.json['entity']['id']

### Edit a wikibase entity

In [17]:
def edit_entity(session, qid, data):
    token = get_csrf_token(session)
    
    params = {
        'id': qid,
        'action': 'wbeditentity',
        'format': 'json',
        'data': json.dumps(data),
        'token': token
    }
    r1 = session.post(WIKIBASE_API, data=params)
    r1.json = r1.json()
    
    # raise when edit failed
    if 'error' in r1.json.keys():
        raise WBAPIException(r1.json['error'])

    return r1.json['entity']['id']


### Read entity by title

In [2]:
def read_entity_by_title(session, title):
    """Reads the Qid of an entity."""
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'search': title,
        'language': 'en',
        'type': 'item',
        'limit': 1
    }
    r1 = session.post(WIKIBASE_API, data=params)
    r1.json = r1.json()
    qid = None
    if 'search' in r1.json.keys():
        if len(r1.json['search']) > 0:
            qid = r1.json['search'][0]['id']
    return qid